In [1]:
import pandas as pd
import plotly.express as px
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import xgboost as xgb
import os

from sklearn import tree
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV, learning_curve



/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.25.2
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
network_attack_1 = pd.read_csv('../dataset/Network datatset/csv/attack_1.csv', sep=",")
network_attack_2 = pd.read_csv('../dataset/Network datatset/csv/attack_2.csv', sep=",")
network_attack_3 = pd.read_csv('../dataset/Network datatset/csv/attack_3.csv', sep=",")
network_attack_4 = pd.read_csv('../dataset/Network datatset/csv/attack_4.csv', sep=",")
network_normal = pd.read_csv('../dataset/Network datatset/csv/normal.csv', sep=",")

df_att1 = pd.read_csv("../dataset/Physical dataset/phy_att_1.csv", encoding = 'UTF-16', sep="\t")
df_att2 = pd.read_csv("../dataset/Physical dataset/phy_att_2.csv", encoding = 'UTF-16', sep="\t")
df_att3 = pd.read_csv("../dataset/Physical dataset/phy_att_3.csv", encoding = 'UTF-16', sep="\t")
df_att4 = pd.read_csv("../dataset/Physical dataset/phy_att_4.csv", sep=",")
df_norm = pd.read_csv("../dataset/Physical dataset/phy_norm.csv", encoding = 'UTF-16', sep="\t")

df_att1.name = "df_att1"
df_att2.name = "df_att2"
df_att3.name = "df_att3"
df_att4.name = "df_att4"
df_norm.name = "df_norm"
network_attack_1.name = "network_attack_1"
network_attack_2.name = "network_attack_2"
network_attack_3.name = "network_attack_3"
network_attack_4.name = "network_attack_4"
network_normal.name = "network_normal"

In [3]:
df_all = [df_att1, df_att2, df_att3, df_att4, df_norm, network_attack_1, network_attack_2, network_attack_3, network_attack_4, network_normal]
def df_all_fn(func):
    df_all = [df_att1, df_att2, df_att3, df_att4, df_norm, network_attack_1, network_attack_2, network_attack_3, network_attack_4, network_normal]
    for df in df_all:
        print(df.name)
        print("\n" + df.name + " -------------------------------------")
        func(df)

df_all_net = [network_attack_1, network_attack_2, network_attack_3, network_attack_4, network_normal]
def df_all_net_fn(func):
    df_all_net = [network_attack_1, network_attack_2, network_attack_3, network_attack_4, network_normal]
    for df in df_all_net:
        print(df.name)
        print("\n" + df.name + " -------------------------------------")
        func(df)

In [4]:
# def transform

# df_net_att2['Time'] = pd.to_datetime(df_net_att2['Time'], format='mixed')
# df_phys_att2['Time'] = pd.to_datetime(df_phys_att2['Time'], format='mixed')
# df_net_att2['Time'] = df_net_att2['Time'].dt.floor('s')

In [5]:
for i, df in enumerate(df_all):
    df_all[i] = df_all[i].rename(columns = {'Time':'time'}, inplace= True)

In [6]:
network_attack_1.columns = network_attack_1.columns.str.strip()
network_attack_2.columns = network_attack_2.columns.str.strip()
network_attack_3.columns = network_attack_3.columns.str.strip()
network_attack_4.columns = network_attack_4.columns.str.strip()
network_normal.columns = network_normal.columns.str.strip()

df_att1.columns = df_att1.columns.str.strip()
df_att2.columns = df_att2.columns.str.strip()
df_att3.columns = df_att3.columns.str.strip()
df_att4.columns = df_att4.columns.str.strip()
df_norm.columns = df_norm.columns.str.strip()

In [7]:
network_attack_1.head

<bound method NDFrame.head of                                time              mac_s              mac_d  \
0        2021-04-09 18:23:28.385003  74:46:a0:bd:a7:1b  0a:fe:ec:47:74:fb   
1        2021-04-09 18:23:28.385005  74:46:a0:bd:a7:1b  e6:3f:ac:c9:a8:8c   
2        2021-04-09 18:23:28.385006  74:46:a0:bd:a7:1b  fa:00:bc:90:d7:fa   
3        2021-04-09 18:23:28.385484  0a:fe:ec:47:74:fb  74:46:a0:bd:a7:1b   
4        2021-04-09 18:23:28.385486  fa:00:bc:90:d7:fa  74:46:a0:bd:a7:1b   
...                             ...                ...                ...   
5527404  2021-04-09 19:03:47.660871  74:46:a0:bd:a7:1b  0a:fe:ec:47:74:fb   
5527405  2021-04-09 19:03:47.660871  74:46:a0:bd:a7:1b  00:80:f4:03:fb:12   
5527406  2021-04-09 19:03:47.661288  fa:00:bc:90:d7:fa  74:46:a0:bd:a7:1b   
5527407  2021-04-09 19:03:47.661290  e6:3f:ac:c9:a8:8c  74:46:a0:bd:a7:1b   
5527408  2021-04-09 19:03:47.661291  0a:fe:ec:47:74:fb  74:46:a0:bd:a7:1b   

                 ip_s          ip_d    sport 

In [8]:
# As there is too much na and [0] values, the column isn't representative so we drop it
def drop_modbus_response(df):
    del df['modbus_response']
    print("modbus_response dropped for", df.name)

df_all_net_fn(drop_modbus_response)

network_attack_1

network_attack_1 -------------------------------------
modbus_response dropped for network_attack_1
network_attack_2

network_attack_2 -------------------------------------
modbus_response dropped for network_attack_2
network_attack_3

network_attack_3 -------------------------------------
modbus_response dropped for network_attack_3
network_attack_4

network_attack_4 -------------------------------------
modbus_response dropped for network_attack_4
network_normal

network_normal -------------------------------------
modbus_response dropped for network_normal


In [9]:
# Drop duplicates without counting the time column, that is unique
# There is no duplicate when only ignoring the "time" column

network_attack_1.drop_duplicates(subset=network_attack_1.columns.difference(['time']), inplace=True)
network_attack_2.drop_duplicates(subset=network_attack_2.columns.difference(['time']), inplace=True)
network_attack_3.drop_duplicates(subset=network_attack_3.columns.difference(['time']), inplace=True)
network_attack_4.drop_duplicates(subset=network_attack_4.columns.difference(['time']), inplace=True)
network_normal.drop_duplicates(subset=network_normal.columns.difference(['time']), inplace=True)


In [10]:
df_att1.drop_duplicates(subset=df_att1.columns.difference(['time']), inplace=True)
df_att2.drop_duplicates(subset=df_att2.columns.difference(['time']), inplace=True)
df_att3.drop_duplicates(subset=df_att3.columns.difference(['time']), inplace=True)
df_att4.drop_duplicates(subset=df_att4.columns.difference(['time']), inplace=True)
df_norm.drop_duplicates(subset=df_norm.columns.difference(['time']), inplace=True)

In [11]:
# From further informations, we can see that modbus_fn is always NaN for these attacks, so we replace the NaN by "Not specified".
def change_column(col_name, value_to_replace):
    df_all_net = [network_attack_1, network_attack_2, network_attack_3, network_attack_4, network_normal]
    for df in df_all_net:
        if value_to_replace == "median":
            df[col_name] = df[col_name].fillna(df[col_name].median())
        else:
            df[col_name] = df[col_name].fillna(value_to_replace)

change_column('modbus_fn', "Not specified")

In [12]:
# We do the same for other columns
change_column('ip_d', "Not specified")
change_column('ip_s', "Not specified")

# We replace by the median
change_column('sport', "median")
change_column('dport', "median")
change_column('n_pkt_src', "median")
change_column('n_pkt_dst', "median")
change_column('flags', "median")

In [13]:
# Finally, as we don't want NA values in our dataset (some algorithms don't support them), we will drop them and check if there is still all attacks available
# We don't care about sport and dport columns, as we are not going to use them later
network_attack_1.dropna(how='any', inplace=True, subset=network_attack_1.columns.difference(['sport', 'dport']))
network_attack_2.dropna(how='any', inplace=True, subset=network_attack_1.columns.difference(['sport', 'dport']))
network_attack_3.dropna(how='any', inplace=True, subset=network_attack_1.columns.difference(['sport', 'dport']))
network_attack_4.dropna(how='any', inplace=True, subset=network_attack_1.columns.difference(['sport', 'dport']))
network_normal.dropna(how='any', inplace=True, subset=network_attack_1.columns.difference(['sport', 'dport']))

df_att1.dropna(how='any', inplace=True, subset=df_att1.columns.difference(['sport', 'dport']))
df_att2.dropna(how='any', inplace=True, subset=df_att2.columns.difference(['sport', 'dport']))
df_att3.dropna(how='any', inplace=True, subset=df_att3.columns.difference(['sport', 'dport']))
df_att4.dropna(how='any', inplace=True, subset=df_att4.columns.difference(['sport', 'dport']))
df_norm.dropna(how='any', inplace=True, subset=df_norm.columns.difference(['sport', 'dport']))
print(network_attack_2["label"].unique())

['normal' 'anomaly' 'scan' 'DoS' 'physical fault' 'MITM']


In [14]:
df_all = [df_att1, df_att2, df_att3, df_att4, df_norm, network_attack_1, network_attack_2, network_attack_3, network_attack_4, network_normal]
data_fusion_phys = pd.concat([df_att1, df_att2, df_att3, df_att4, df_norm])
data_fusion_net = pd.concat([network_attack_1, network_attack_2, network_attack_3, network_attack_4, network_normal])
data_fusion = pd.merge(data_fusion_phys, data_fusion_net, on='time', how='right')

In [15]:
del data_fusion['time']

In [16]:
# We normalize the numerical data that don't act as a category. For example, a sport is a numerical category but nb_pkt_src is purely numerical

scaler = MinMaxScaler()
data_fusion_numerical = data_fusion[['size', 'n_pkt_src', 'n_pkt_dst']]
data_fusion_numerical = pd.DataFrame(scaler.fit_transform(data_fusion_numerical), columns=data_fusion_numerical.columns)

In [17]:
# One-hot encoding the categorical data and numerical data that acts as a category
data_fusion_one_hot_encoded = data_fusion
# We don't care about label for now, we are going to add it later
# We also delete sport and dport because after the one-hot encoding, we get 64000 more columns which creates a dataset of 70 GB
data_fusion_one_hot_encoded = pd.get_dummies(data_fusion_one_hot_encoded[['mac_s', 'mac_d', 'ip_s', 'ip_d', 'proto', 'flags', 'size', 'modbus_fn']], columns=['mac_s', 'mac_d', 'ip_s', 'ip_d', 'proto', 'flags', 'size', 'modbus_fn'])

In [18]:
data_fusion_one_hot_encoded_and_normalized = pd.concat([data_fusion_numerical, data_fusion_one_hot_encoded], axis=1)
# Datasets provided in SVM should not contain one-hot encoded data
data_fusion_svm = pd.concat([data_fusion_numerical, data_fusion.copy().drop(columns=data_fusion_numerical.columns)], axis=1)

In [19]:
Y= data_fusion['label']
X= data_fusion_one_hot_encoded_and_normalized

# Y_train_SVM and Y_test_SVM contains the label not one-hot encoded because it doesn't needs it
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
Y_test_svm=np.ravel(Y_test) 
Y_train_svm=np.ravel(Y_train)

Y_train= pd.get_dummies(Y_train, columns=['label'])
Y_test= pd.get_dummies(Y_test, columns=['label'])
DataSet={"X_train": X_train, "X_test":X_test, "Y_train":Y_train, "Y_test":Y_test,'Y_train_svm':Y_train_svm,'Y_test_svm':Y_test_svm}

# Applying some algorithms

In [20]:
def one_hot_to_label(one_hot_array):
    return np.argmax(one_hot_array, axis=1)

def plot_confusion_matrix(y_true, y_pred, classes, is_svm=False):
    if not is_svm:
        y_true = one_hot_to_label(y_true)
        y_pred = one_hot_to_label(y_pred)

    cm = confusion_matrix(y_true, y_pred)
    
    plt.figure(figsize=(len(classes), len(classes)))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", xticklabels=classes, yticklabels=classes)
    
    plt.title('Matrice de Confusion')
    plt.xlabel('Valeurs Prédites')
    plt.ylabel('Valeurs Réelles')
    return plt
    
def plot_learning_curve_f1(estimator, X, y, cv=None, n_jobs=None, train_sizes=np.linspace(.1, 1.0, 5)):
    plt.figure()
    plt.title("Courbes d'apprentissage")
    plt.xlabel("Taille de l'échantillon d'apprentissage")
    plt.ylabel("F1 score")
    # Utiliser la fonction learning_curve avec le paramètre scoring='f1_micro' pour calculer le f1 score moyen sur chaque sous-ensemble
    train_sizes, train_scores, test_scores = learning_curve(estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes, scoring='f1_macro')
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()
    plt.fill_between(train_sizes, train_scores_mean - train_scores_std, train_scores_mean + train_scores_std, alpha=0.1, color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std, test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r", label="F1 score sur l'ensemble d'apprentissage")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g", label="F1 score sur l'ensemble de validation croisée")
    plt.legend(loc="best")
    return plt

In [21]:
import time
import math

def Models(model, DataSet):
    resultat_model = {}
    
    start_fit = time.time()

    if model == 'cart':
        parameters = {'max_depth': [None, 10, 20, 30], 'min_samples_split': [2, 5, 10]}
        Model_cart = GridSearchCV(tree.DecisionTreeClassifier(), parameters, cv=5)
        Model_cart.fit(DataSet['X_train'], DataSet['Y_train'])
        resultat_model['cart'] = Model_cart.best_estimator_

    elif model == 'xgb':
        parameters = {'max_depth': [3, 5, 7], 'learning_rate': [0.01, 0.1, 0.2], 'n_estimators': [50, 100, 200]}
        Model_XGBC = GridSearchCV(xgb.XGBClassifier(), parameters, cv=5)
        Model_XGBC.fit(DataSet['X_train'], DataSet['Y_train'])
        resultat_model['xgb'] = Model_XGBC.best_estimator_

    elif model == 'knn':
        parameters = {'n_neighbors': [3, 5, 7]}
        Model_knn = GridSearchCV(KNeighborsClassifier(), parameters, cv=5)
        Model_knn.fit(DataSet['X_train'], DataSet['Y_train'])
        resultat_model['knn'] = Model_knn.best_estimator_

    elif model == 'svm':
        Model_svm =SVC(kernel='poly', degree=5)
        Model_svm.fit(DataSet['X_train'],DataSet["Y_train_svm"])
        resultat_model['svm']=Model_svm
        
        
    elif model == 'mlp':
        parameters = {'hidden_layer_sizes': [(100,), (200,)]}
        Model_mlp = GridSearchCV(MLPClassifier(), parameters, cv=5)
        Model_mlp.fit(DataSet['X_train'], DataSet['Y_train'])
        resultat_model["mlp"] = Model_mlp.best_estimator_

    elif model == 'randomForest':
        parameters = {'n_estimators': [50, 100, 200], 'max_depth': [None, 10, 20, 30]}
        Model_rf = GridSearchCV(RandomForestClassifier(), parameters, cv=5)
        Model_rf.fit(DataSet['X_train'], DataSet['Y_train'])
        resultat_model['randomForest'] = Model_rf.best_estimator_

    else:
        print("Le nom du modèle doit être compris dans cette liste : {cart, xgb, knn, svm, mlp, randomForest}")
    end_fit = time.time()
    elapsed_time = math.floor(end_fit - start_fit) + 1
    return resultat_model, elapsed_time


In [22]:
import json

if not os.path.exists('results'):
    os.makedirs('results/concat/')

def evaluate_models(DataSet):
        
    sizes = [100, 1000, 10000, 100000]
    label=['Label_DoS', 'Label_MITM', 'Label_normal', 'Label_physical fault', 'Label_anomaly', 'Label_scan']

    performance_results = {} # dictionnaire pour stocker les performances pour chaque modèle et chaque taille

    for size in sizes:
        subset_X_train = DataSet['X_train'][:size]
        subset_Y_train = DataSet['Y_train'][:size]
        subset_Y_train_svm = DataSet['Y_train_svm'][:size]

        # Création des modèles 
        print('\n')
        print('data SIZE:',size)
        print('\n')
        for model_type in ['cart', 'xgb', 'knn', 'randomForest', 'mlp', 'svm']:
                
            # Entraînez le modèle
            model_dict, fit_time = Models(model_type, {'X_train': subset_X_train, 'Y_train': subset_Y_train,'Y_train_svm': subset_Y_train_svm})
            model = model_dict[model_type]  # Accédez au modèle spécifique dans le dictionnaire
            
            #prédictions sur l'ensemble de test
            predictions = model.predict(DataSet['X_test'])

            start_predict = time.time()
            predictions = model.predict(DataSet['X_test'])
            end_predict = time.time()
            pred_time = math.floor(end_predict - start_predict) + 1
            
            print("FIT TIME")
            print(fit_time)
            print("PRED TIME")
            print(pred_time)

            print(f"{model_type}_size_{size}")

            image_path = "results/concat/all/" + str(size) + "/" + model_type
            if not os.path.exists(image_path):
                os.makedirs(image_path)

            time_results = {
                "fit_time": fit_time,
                "pred_time": pred_time
            }
            
            with open(image_path + "/time.json", "w") as outfile: 
                json.dump(time_results, outfile)

            # Évaluez les performances  des modèles
            if model_type=="svm":
                
                accuracy = accuracy_score(DataSet['Y_test_svm'], predictions)
                print(f"{model_type}: Accuracy = {accuracy}")
                print('\n')

                report = classification_report(DataSet['Y_test_svm'], predictions, target_names=label, output_dict=True)
                sns.heatmap(pd.DataFrame(report).iloc[:-1, :].T, annot=True)
                plt.savefig(image_path + "/classification_report.png")

                plot_confusion_matrix(DataSet['Y_test_svm'], predictions, label, is_svm=True)
                plt.savefig(image_path + "/confusion_matrix.png")
                plt.show()
                
                plot_learning_curve_f1(model, subset_X_train, subset_Y_train_svm, cv=5)
                plt.savefig(image_path + "/f1_curve.png")
                plt.show()
            
            else:
                
                accuracy = accuracy_score(DataSet['Y_test'], predictions)
                print(f"{model_type}: Accuracy = {accuracy}")

                report = classification_report(DataSet['Y_test'], predictions, target_names=label, output_dict=True)
                sns.heatmap(pd.DataFrame(report).iloc[:-1, :].T, annot=True)
                plt.savefig(image_path + "/classification_report.png")
                
                plot_confusion_matrix(DataSet['Y_test'].to_numpy(),predictions, label)
                plt.savefig(image_path + "/confusion_matrix.png")
                plt.show()
                
                plot_learning_curve_f1(model, subset_X_train, subset_Y_train, cv=5)
                plt.savefig(image_path + "/f1_curve.png")
                plt.show()
        
                

            # Stockez les résultats dans le dictionnaire
            key = f"{model_type}_size_{size}"
            performance_results[key]=model_dict
            
    return performance_results


In [2]:
evaluate_models(DataSet)

NameError: name 'evaluate_models' is not defined